In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import RobertaTokenizer
import torch
from tqdm import tqdm
torch.cuda.empty_cache()
import numpy as np
from sklearn.metrics import f1_score

In [2]:
labels_dict = {'NEUTRAL':0, 'OPINIONATED':1}
df = pd.read_csv('../data/cleaned_subj_polar__.csv', encoding='ISO-8859-1')
df['label'] = df['subjectivity'].apply(lambda x: labels_dict[x])
print(len(df))
df.head()

1200


,Unnamed: 0,text,subjectivity,polarity,label
0,0,Love our President,OPINIONATED,POSITIVE,1
1,1,"Yes, same guy!!!!",NEUTRAL,NaN,0
2,2,Thank God we Parlerðâï¸ðºð¸.I'm grat...,OPINIONATED,POSITIVE,1
3,3,"This would be the best case scenario, Wednesda...",NEUTRAL,NaN,0
4,4,Sharpton is as non-partisan as lions are veget...,NEUTRAL,NaN,0


In [3]:
text = list(df.text)
labels = list(df.label)
train_texts, val_texts, train_labels, val_labels = train_test_split(text, labels, test_size=.15, stratify=labels)

In [4]:
from torch.utils.data import TensorDataset
from transformers import BertTokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [6]:
train_encode = tokenizer.batch_encode_plus(
    train_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)

valid_encode = tokenizer.batch_encode_plus(
    val_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)
valid_input = valid_encode['input_ids']
valid_attention = valid_encode['attention_mask']
valid_labels = torch.tensor(val_labels)

input_ids = train_encode['input_ids']
attention_mask = train_encode['attention_mask']
labels = torch.tensor(train_labels)

In [7]:
train_data = TensorDataset(input_ids,
                          attention_mask,
                          labels)

valid_data = TensorDataset(valid_input,
                          valid_attention,
                          valid_labels)

In [10]:
from transformers import BertForSequenceClassification

#Finetuning pretrained BERT model with objective -> Sentence Classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 8

train_dataloader = DataLoader(train_data,
                              sampler = RandomSampler(train_data),
                              batch_size = BATCH_SIZE)
valid_dataloader = DataLoader(valid_data,
                              sampler = SequentialSampler(valid_data),
                              batch_size = BATCH_SIZE)


In [12]:
from transformers import AdamW, get_cosine_schedule_with_warmup

#Works better than torch.optim.Adam
optimizer = AdamW(model.parameters(),
                           lr=1e-5,
                           eps=1e-6,
                           weight_decay=1e-1)
#chosen after setting = 3 (f1 -> 0.71), 5 (f1 -> 0.77) but acpc for many classes is 0
#10 Epochs helps with generalization
EPOCHS = 10 

#Works better than torch.optim.CosineAnnealingLR and transformers.get_linear_schedule_with_warmup
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [13]:
def f1(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [14]:
#Per Class Accuracy
def acpc(preds, labels):
    code_dict = {val:key for key,val in labels_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for idx in np.unique(labels_flat):
        y_hat = preds_flat[labels_flat==idx]
        y = labels_flat[labels_flat==idx]
        print(f'Class: {code_dict[idx]}')
        print(f'Accuracy: {len(y_hat[y_hat==idx])/ len(y)}\n')

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [16]:
def evaluate(valid_dataloader):
    model.eval()

    total_eval_loss = 0
    y_hat, y = [], []

    for batch in valid_dataloader:
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids' : batch[0],
                'attention_mask': batch[1],
                'labels' : batch[2]
                }
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        y_hat.append(logits)
        y.append(label_ids)

    avg_eval_loss = total_eval_loss/len(valid_dataloader) 

    y_hat = np.concatenate(y_hat, axis=0)
    y = np.concatenate(y, axis=0)
            
    return avg_eval_loss, y_hat, y

In [17]:
vloss = []
score = []
for epoch in tqdm(range(1, EPOCHS+1)):
    model.train()

    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids' : batch[0],
                  'attention_mask': batch[1],
                  'labels' : batch[2]
                  }
        outputs = model(**inputs)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f'./bert subjectivity/BERT_subjectivity_epoch_{epoch}.pt')

    tqdm.write(f'\nEpoch {epoch}')

    avg_training_loss = total_loss/len(train_dataloader)            
    tqdm.write(f'Training loss: {avg_training_loss}')

    val_loss, predictions, actual = evaluate(valid_dataloader)
    score_f1 = f1(predictions, actual)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {score_f1}')
    vloss.append(val_loss)
    score.append(score_f1)

Epoch 1:  51%|█████████████████████████▉                         | 65/128 [00:41<00:40,  1.57it/s, training_loss=0.171]


  0%|                                                                                           | 0/10 [01:24<?, ?it/s]


Epoch 1
Training loss: 0.5357604565797374


Epoch 2:   0%|                                                                                 | 0/128 [00:00<?, ?it/s]

Validation loss: 0.3679007836010145
F1 Score (Weighted): 0.8444252376836646


...


Epoch 10:  52%|█████████████████████████▊                        | 66/128 [00:43<00:40,  1.53it/s, training_loss=0.001]


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [14:35<01:28, 88.24s/it]


Epoch 10
Training loss: 0.004489393881158321


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [14:38<00:00, 87.89s/it]

Validation loss: 0.4262749595446107
F1 Score (Weighted): 0.8944151153098084


In [18]:
for i, pair in enumerate(zip(vloss, score)):
    print(i, pair[0], pair[1])

0 0.3679007836010145 0.8444252376836646
1 0.3249636835054211 0.8827216034252738
2 0.31451235168977926 0.8777173913043479
3 0.44537161760356114 0.8710722182429698
4 0.3804779073385441 0.8944151153098084
5 0.40312755927848426 0.8888751697740461
6 0.41395653873358085 0.8944151153098084
7 0.42358328452657745 0.8944151153098084
8 0.425832158146912 0.8944151153098084
9 0.4262749595446107 0.8944151153098084


In [19]:
model.to(device)
model.load_state_dict(torch.load('./bert subjectivity/BERT_subjectivity_epoch_5.pt', map_location=torch.device('cpu')))

_, predictions, actual = evaluate(valid_dataloader)
print(acpc(predictions, actual))
labs = {v:k for k,v in labels_dict.items()}
for i in range (len(predictions)):
    print(labs[np.argmax(predictions[i])], val_texts[i])
    print()

Class: NEUTRAL
Accuracy: 0.9111111111111111

Class: OPINIONATED
Accuracy: 0.8777777777777778

None
NEUTRAL Watch Perdue lose 30,000 votes in real time

NEUTRAL Yes We need to keep repeating this truth

OPINIONATED Essentially Obama and Clinton are guilty of war crimes should be put to death! How many people did they kill! That be wonderful if we had trials!

OPINIONATED Such bad actingð¥¸

OPINIONATED Absolutely gorgeous photo of a HUGE number of Patriots!

OPINIONATED Our country is worth fighting for. Let's make all of our veterans proud.

OPINIONATED HOKEY POKEY THIS A FUNKY

OPINIONATED This is a great beer. Drink one and youâre feeling good

NEUTRAL Iâm with you on this. Been watching from the beginning. Now they are saying two have been shot. There are videos of the one woman. Even though we the people entered the capital building. No one was tearing shit up. No destruction no vandalism or fires but the news is making it out that today was like blm an antifa do.

OPINIONATED